Create model

In [1]:
yellow_follow = 'C:/Users/caspe/Desktop/yellow/lib/'
import sys; sys.path.append(yellow_follow) 

import os
import numpy as np
import pandas as pd
import ml_utils
from sqlalchemy import create_engine
from sklearn.model_selection import GridSearchCV

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Dropout, Conv2D, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.constraints import max_norm

np.set_printoptions(suppress=True)
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
folder = "C:/Users/caspe/Desktop/Paper_6_Generative/"

# Load data
X_train = np.load(folder + "X_train.npy").transpose(0, 2, 3, 1)
y_train = np.load(folder + "y_train.npy")[..., np.newaxis]
X_test = np.load(folder + "X_test.npy").transpose(0, 2, 3, 1)
y_test = np.load(folder + "y_test.npy")[..., np.newaxis]

In [3]:
def define_model(shape, name, activation='relu', kernel_initializer='normal', dropout=0.2, maxnorm=4, sizes=[32, 64, 96]):

    model_input = Input(shape=shape, name=name)
    model = Conv2D(sizes[0],
        kernel_size=3,
        padding='same',
        activation=activation,
        kernel_initializer=kernel_initializer,
        kernel_constraint=max_norm(maxnorm),
        bias_constraint=max_norm(maxnorm),
    )(model_input)
    model = BatchNormalization()(model)
    model = Dropout(dropout)(model)

    model = Conv2D(sizes[1],
        kernel_size=3,
        padding='same',
        activation=activation,
        kernel_initializer=kernel_initializer,
        kernel_constraint=max_norm(maxnorm),
        bias_constraint=max_norm(maxnorm),
    )(model)
    model = BatchNormalization()(model)
    model = Dropout(dropout)(model)

    model = Conv2D(sizes[2],
        kernel_size=3,
        padding='same',
        activation=activation,
        kernel_initializer=kernel_initializer,
        kernel_constraint=max_norm(maxnorm),
        bias_constraint=max_norm(maxnorm),
    )(model)
    model = BatchNormalization()(model)
    model = Dropout(dropout)(model)

    output = Conv2D(1, kernel_size=3, padding='same', activation='relu', kernel_initializer=kernel_initializer)(model)

    return Model(inputs=[model_input], outputs=output)

In [ ]:
model = define_model(X_train.shape[1:], "Generative", dropout=0.0)
model.summary()

In [ ]:
epochs = 50
lr = 0.001
bs = 512

model.compile(
    optimizer=Adam(
            learning_rate=lr,
            name="Adam",
    ),
    loss='log_cosh',
    metrics=[
        'mse',
        'mae',
        'log_cosh',
    ]
)

model.fit(
    x=X_train,
    y=y_train,
    epochs=epochs,
    verbose=1,
    batch_size=bs,
    validation_split=0.2,
)

loss, mse, mae, log_cosh = model.evaluate(X_test, y_test, verbose=0)
print(f"Mean Square Error:      {round(mse, 3)}")
print(f"Mean Absolute Error:    {round(mae, 3)}")
print(f"log_cosh:               {round(log_cosh, 3)}")
print("")

In [ ]:
from matplotlib import pyplot

pyplot.plot(mode.history['accuracy'], label='train')
pyplot.plot(model.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
pred = model.predict(X_test)

In [ ]:
gimg = 2001
import matplotlib.pyplot as plt

def to_8bit(arr, arrmax=None):
    if arrmax == None:
        red = (arr * 255 / y_test.max()).astype('uint8')
    else:
        red = (arr * 255 / arrmax).astype('uint8')
    if len(red.shape) == 3:
        return red[:, :, 0]
    return red

In [ ]:
plt.imshow(to_8bit(pred[gimg]), interpolation='none')
plt.show()

In [ ]:
plt.imshow(to_8bit(y_test[gimg]), interpolation='none')
plt.show()

In [ ]:
plt.imshow(to_8bit(X_test[gimg][:, :, 0], 1), interpolation='none')
plt.show()